In [49]:
# includes
from PIL import Image
from skimage import io, transform

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import torch.optim as optim

In [50]:
# constansts

project_path = "/home/krys/POLIBUDA/Patterns_database/"


dots_path = project_path + "dots/dot"
paisley_path = project_path + "paisley/paisley"
qua_path = project_path + "qua/qua"
square_path = project_path + "square/sq"
strips_path = project_path + "strips/pas"
traingles_path = project_path + "triangles/tr"

dots_target = torch.tensor([[float(1), float(0), float(0), float(0), float(0), float(0)]])
paisley_target = torch.tensor([[float(0), float(1), float(0), float(0), float(0), float(0)]])
qua_target = torch.tensor([[float(0), float(0), float(1), float(0), float(0), float(0)]])
square_target = torch.tensor([[float(0), float(0), float(0), float(1), float(0), float(0)]])
strips_target = torch.tensor([[float(0), float(0), float(0), float(0), float(1), float(0)]])
traingles_target = torch.tensor([[float(0), float(0), float(0), float(0), float(0), float(1)]])

paths = [
    dots_path,
    paisley_path,
    qua_path,
    square_path,
    strips_path,
    traingles_path
]

targets = [
    dots_target,
    paisley_target,
    qua_target,
    square_target,
    strips_target,
    traingles_target
]


def getImageTensor(path, idx):
    
    # loading image from local memory
    
    i_path = path + str(idx) + ".jpg"
    image = io.imread(i_path)
    
    return image
    

# transforms image to grayscale tensor
def myTransform(image):

    transformedImage =  tv.transforms.Compose([
                        tv.transforms.ToPILImage(),
                        tv.transforms.Grayscale(num_output_channels=1),
                        tv.transforms.ToTensor()
                      ])(image)
    
    transformedImage = transformedImage[0]

    return transformedImage

In [51]:
# nn model using pytorch schematic
class PatternRecognitionNet(nn.Module):

    def __init__(self):
        super(PatternRecognitionNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 73 * 73, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 6)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
# creting net, lossfunction, optimizer    
pattNet = PatternRecognitionNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01) # check learning rate

In [52]:
# training function
def train(image, target):
    
    optimizer.zero_grad()   # zero the gradient buffers
    input = image.view(-1,1,300,300)
    output = pattNet(input)
    print("Target: ",target)
    print("Output: ",output)
    loss = criterion(output, target)
    print("Loss calculation: " , loss)
    loss.backward()
    optimizer.step()    # Does the update

In [53]:
idx = 1
while(1):
    try:
        
        for k in range(len(paths)):    
            print("============================================================")
            print("Training : " + str(idx) + " : " + str(k+1) + "/6")
            print("\n")
            image = getImageTensor(paths[k], idx)
            t_image = myTransform(image)
            train(t_image, targets[k])
            print("\n")
        
        
        idx = idx + 1
        
    except Exception as e:
        print(e)
        break 

Training : 1 : 1/6


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[ 0.0709,  0.0951, -0.0070,  0.0140,  0.0209, -0.1501]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1492, grad_fn=<MseLossBackward>)


Training : 1 : 2/6


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[ 0.0717,  0.0902, -0.0034,  0.0045,  0.0230, -0.1498]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1426, grad_fn=<MseLossBackward>)


Training : 1 : 3/6


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[ 0.0695,  0.0955, -0.0053,  0.0158,  0.0215, -0.1571]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1750, grad_fn=<MseLossBackward>)


Training : 1 : 4/6


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[ 0.0621,  0.0653,  0.0164, -0.0115,  0.0176, -0.1259]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1746, grad_fn=<MseLossBackward>)


Training : 1 : 5/6


Target:  tensor([[0., 0., 0., 0., 1., 0.]])